In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from functools import reduce
import statsmodels.formula.api as smf
from pkg import pearson_corr, nakagawa_r2
import os
os.chdir("/Users/caropark/FAO_ag_check_code/")

In [2]:
gdp = pd.read_csv("./data/wb_gdp_per_cap.csv").rename({'value': 'gdp'}, axis=1)
gdp = gdp.groupby(['iso_a3']).agg(avg_gdp = ('gdp', 'mean')).reset_index()

flag = pd.read_csv("./data/faostat_all_flags.csv").rename({"Year": 'year'}, axis=1)
flag = flag.groupby(["iso_a3", 'cropname'])[['year']].agg(flag_sum = ('year', 'count')).reset_index()

modis = pd.read_csv("./data/modis_vars.csv")[['total_harvarea', 'cropland_fraction', 'cropname', 'iso_a3']]

corrupt = pd.read_csv("./data/corruption.csv").filter(regex= "CPI|Country|ISO3")
corrupt = pd.melt(corrupt, id_vars=['Country', "ISO3"]).drop("variable", axis=1)
corrupt.columns= ["corr_country", "iso_a3", "corrupt_score"]
country_key= pd.read_csv("./data/country_key.csv")[['iso_a3']]
corrupt = corrupt.merge(country_key, how="left", on="iso_a3").iloc[:,1:].groupby("iso_a3").mean().reset_index()

clim = (pd.read_csv("./data/dt_clim_vars.csv").groupby(['iso_a3', 'cropname'])[['sm', 'tmax']]
        .agg(avg_sm = ('sm', 'mean'), avg_tmax= ('tmax', 'mean')).reset_index())

In [3]:
yields = pd.read_csv("./data/yield_comparison.csv")[["cropname", "year", "yield", "csif", "whichlag", "country", "iso_a3"]]
yield_r2 = yields.groupby(['iso_a3', 'cropname', 'whichlag']).apply(lambda group: pd.Series({"r2": pearson_corr(group, x="csif",y="yield")**2})).reset_index()
yield_r2 = yield_r2[yield_r2['r2']!=1]


In [4]:
def flexible_merge(left, right):
    keys = ["iso_a3"]
    if "cropname" in left.columns and "cropname" in right.columns:
        keys.append("cropname")
    return pd.merge(left, right, on=keys, how="outer")

dfs = [yield_r2, gdp, flag, modis, corrupt, clim]

merged = reduce(flexible_merge, dfs).dropna(subset=["r2","avg_sm"]).drop_duplicates().reset_index(drop=True)
merged["flag_sum"] = merged["flag_sum"].fillna(0)

coeffs = ["avg_gdp","flag_sum","total_harvarea","cropland_fraction","avg_sm","avg_tmax", "whichlag"]
merged = merged.dropna(subset=coeffs.copy().extend(["r2", "cropname"])).reset_index(drop=True)


In [5]:
merged[coeffs[:-1]].agg(['min', 'max'])
scaled = merged[coeffs[:-1]].apply(lambda x: (x - x.mean()) / x.std()).add_suffix("_z")
merged = merged.drop(columns=coeffs[:-1]).join(scaled)
merged

,iso_a3,cropname,whichlag,r2,corrupt_score,avg_gdp_z,flag_sum_z,total_harvarea_z,cropland_fraction_z,avg_sm_z,avg_tmax_z
0,AFG,Barley,yield_dt,0.149151,13.333333,-0.629326,-0.657075,-0.159908,-0.739775,-0.907513,-1.072067
1,AFG,Maize,yield_lag,0.162829,13.333333,-0.629326,-0.657075,-0.171567,-0.739775,-1.963675,0.444760
2,AFG,Millet,yield_lead,0.079590,13.333333,-0.629326,0.856813,-0.200033,-0.740321,-2.216612,0.248583
3,AFG,Potatoes,yield_lag,0.003609,13.333333,-0.629326,-0.657075,-0.114451,-0.739775,-1.781418,0.289542
4,AFG,Pulses nes,yield_lag,0.080130,13.333333,-0.629326,2.095449,-0.209874,-0.740575,-1.840474,0.618328
...,...,...,...,...,...,...,...,...,...,...,...
1710,ZWE,Sorghum,yield_dt,0.060107,21.888889,-0.581836,-0.519449,-0.158674,-0.726466,-0.909425,0.181313
1711,ZWE,Soybeans,yield_lag,0.102241,21.888889,-0.581836,-0.657075,-0.190814,-0.737172,-0.598081,0.211830
1712,ZWE,Sunflower seed,yield_lag,0.047835,21.888889,-0.581836,-0.657075,-0.200897,-0.680318,-0.077549,0.165034
1713,ZWE,Sweet potatoes,yield_dt,0.000003,21.888889,-0.581836,1.957823,-0.210882,-0.737326,-1.661319,0.268871


In [6]:
mod_yields_r2_gdp = smf.mixedlm("r2 ~ avg_gdp_z",
                     merged, groups="cropname" )

print(mod_yields_r2_gdp.fit().summary())
nakagawa_r2(mod_yields_r2_gdp.fit())


        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: r2      
No. Observations: 1715    Method:             REML    
No. Groups:       19      Scale:              0.0231  
Min. group size:  43      Log-Likelihood:     772.8533
Max. group size:  145     Converged:          Yes     
Mean group size:  90.3                                
------------------------------------------------------
             Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------
Intercept    0.113    0.010 11.760 0.000  0.094  0.131
avg_gdp_z    0.018    0.004  4.621 0.000  0.010  0.025
cropname Var 0.001    0.004                           

Marginal R² = 0.05828975379387129
Conditional R² = 0.11441748527095114


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [7]:
mod_yields_r2 = smf.mixedlm("r2 ~ avg_gdp_z + flag_sum_z + whichlag + total_harvarea_z + cropland_fraction_z  + avg_sm_z + avg_tmax_z",
                     merged, groups="cropname" )
print(mod_yields_r2.fit().summary())
nakagawa_r2(mod_yields_r2.fit())

              Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    r2      
No. Observations:      1715       Method:                REML    
No. Groups:            19         Scale:                 0.0207  
Min. group size:       43         Log-Likelihood:        844.9399
Max. group size:       145        Converged:             Yes     
Mean group size:       90.3                                      
-----------------------------------------------------------------
                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------
Intercept               0.147    0.008 18.093 0.000  0.131  0.163
whichlag[T.yield_lag]  -0.066    0.009 -7.463 0.000 -0.083 -0.049
whichlag[T.yield_lead] -0.059    0.009 -6.848 0.000 -0.076 -0.042
avg_gdp_z               0.011    0.004  2.685 0.007  0.003  0.019
flag_sum_z             -0.015    0.004 -3.794 0.000 -0.023 -0.007
total_harvarea_z        

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [9]:
res = mod_yields_r2.fit()
params = res.params
conf = res.conf_int()
summary_df = (
    pd.DataFrame({
        "Coef.": params,
        "Std.Err.": res.bse,
        "t": res.tvalues,
        "P>|t|": res.pvalues,
        "CI Lower": conf[0],
        "CI Upper": conf[1]
    })
)

print(summary_df.to_latex(float_format="%.3f"))


\begin{tabular}{lrrrrrr}
\toprule
{} &  Coef. &  Std.Err. &      t &  P>|t| &  CI Lower &  CI Upper \\
\midrule
Intercept              &  0.147 &     0.008 & 18.093 &  0.000 &     0.131 &     0.163 \\
whichlag[T.yield\_lag]  & -0.066 &     0.009 & -7.463 &  0.000 &    -0.083 &    -0.049 \\
whichlag[T.yield\_lead] & -0.059 &     0.009 & -6.848 &  0.000 &    -0.076 &    -0.042 \\
avg\_gdp\_z              &  0.011 &     0.004 &  2.685 &  0.007 &     0.003 &     0.019 \\
flag\_sum\_z             & -0.015 &     0.004 & -3.794 &  0.000 &    -0.023 &    -0.007 \\
total\_harvarea\_z       &  0.017 &     0.004 &  4.633 &  0.000 &     0.010 &     0.024 \\
cropland\_fraction\_z    &  0.020 &     0.004 &  5.319 &  0.000 &     0.013 &     0.028 \\
avg\_sm\_z               & -0.012 &     0.004 & -3.302 &  0.001 &    -0.020 &    -0.005 \\
avg\_tmax\_z             & -0.001 &     0.005 & -0.123 &  0.902 &    -0.011 &     0.009 \\
cropname Var           &  0.034 &     0.016 &  2.156 &  0.031 &     0.003

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/var/folders/k4/2gcvspsj7d7c9hbxlvzv_zrh0000gn/T/ipykernel_55905/2975124526.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summary_df.to_latex(float_format="%.3f"))
